##  Data Preprocessing

Step 2: 

Perform various transformations on our data as required for efficient and effective processing by Machine Learning algorithms based on the type and needs of individual features, identified through the data explorations performed in Step 1 - Data Exploration - Application Data.

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import LabelBinarizer

# Show all of the data in a dataframe
pd.set_option('display.max_columns', None)

In [2]:
# Training Set
application_train = pd.read_csv('data/application_train.csv')
print("Loaded Training Set: {0} rows".format(application_train.shape[0]))

application_test = pd.read_csv('data/application_test.csv')
print("Loaded Training Set: {0} rows".format(application_test.shape[0]))

columns = pd.read_csv('data/HomeCredit_columns_description.csv')

Loaded Training Set: 307511 rows
Loaded Training Set: 48744 rows


In [3]:
# Load the list of features indentified in the exploration step which need preprocessing
non_numeric_features = pd.read_csv('data/tmp/non_numeric_features.csv', header=0, index_col=0, names=["feature"])
numeric_features = pd.read_csv('data/tmp/numeric_features.csv', header=0, index_col=0, names=["feature"])
string_to_bool_features = pd.read_csv('data/tmp/string_to_bool.csv', header=0, index_col=0, names=["feature"])
log_transform_features = pd.read_csv('data/tmp/log_transform.csv', header=0, index_col=0, names=["feature"])

In [4]:
print("Non-Numeric {0}, Numeric {1}, Bool {2}, Log Transform {3}".format(\
        len(non_numeric_features), len(numeric_features), \
        len(string_to_bool_features), len(log_transform_features)))

Non-Numeric 15, Numeric 61, Bool 6, Log Transform 11


In [5]:
'''
Scales a list of numeric features to a range of [0 .. 1] without changing the distribution of the data.  
Accepts a List of column names.  Returns a new dataFrame.
''' 
def scale_features(dataFrame, featureList): 
    dataFrame_transform = pd.DataFrame(data = dataFrame)
    dataFrame_transform[featureList] = scaler.fit_transform(dataFrame_transform[featureList])
    return dataFrame_transform

In [6]:
''' 
Applies an in-place transformation that converts a Y/N field to binary 1/0
'''
def make_bool(dataFrame, featureName): 
    
    lb = LabelBinarizer()
    bool_name = "BOOL_{0}".format(featureName)    
    dataFrame[bool_name] = lb.fit_transform(dataFrame[featureName])
    
    # This seems to work more consistently than df.drop
    # From: https://stackoverflow.com/questions/43838198/df-drop-is-not-working
    del(dataFrame[featureName])    
    #result = dataFrame.drop(featureName, axis=1) 
    
    return dataFrame

In [7]:
''' 
Applies an in-place transformation that converts NaN, Inf and -Inf to numeric values
'''
def make_numeric(dataFrame, featureName): 
        
    return np.nan_to_num(dataFrame[featureName])

In [8]:
'''
Applies an in-place log transformation to numeric features
'''
def log_transform(dataFrame, featureName):     
    
    transformed = dataFrame[featureName].apply(lambda x: np.log(x + 1))
    
    transformed_name = "LOG_{0}".format(featureName)

    print("Transformed Name: {0}".format(transformed_name))
    
    dataFrame[transformed_name] = transformed

    # This seems to work more consistently than df.drop
    # From: https://stackoverflow.com/questions/43838198/df-drop-is-not-working
    del(dataFrame[featureName])
    #result = dataFrame.drop(featureName, axis=1)
    
    return dataFrame

In [9]:
'''
Applies an in-place transformation that offsets all values in a column by the minimum value in the column
'''

def find_offset(dataFrames, featureName): 
    
    offset = 0
    
    for df in dataFrames: 
        if (np.min(df[featureName]) < offset): 
            offset = np.min(df[featureName])

    return abs(offset) + 1
            
            
def offset_negative_values(dataFrame, featureName, offset): 
    print("Offsetting Feature {0} by {1}".format(featureName, offset))
    return dataFrame[featureName].apply(lambda x: (x + offset)), offset

## Preprocessing

### Individual Problematic Features

In [10]:
# DAYS_EMPLOYED

# A bunch of records indicate that the person has worked for ~100 years.
# I'm just going to zero them out, because they're invalid
# NOTE: NaN might ultimately be a better choice

mean_days_employed = np.mean(application_train['DAYS_EMPLOYED'])
application_train['DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].replace({365243: mean_days_employed})
application_test['DAYS_EMPLOYED'] = application_test['DAYS_EMPLOYED'].replace({365243: mean_days_employed})

application_train['FONDKAPREMONT_MODE'] = application_train['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})
application_test['FONDKAPREMONT_MODE'] = application_test['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})

application_train['HOUSETYPE_MODE'] = application_train['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})
application_test['HOUSETYPE_MODE'] = application_test['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})

application_train['WALLSMATERIAL_MODE'] = application_train['WALLSMATERIAL_MODE'].replace({np.nan: "not specified"})
application_test['WALLSMATERIAL_MODE'] = application_test['WALLSMATERIAL_MODE'].replace({np.nan: "not specified"})

### Convert Y/N String Fields to Boolean

In [11]:
# Convert any Y/N string fields to boolean
for feature in string_to_bool_features['feature']: 
    print("Making Boolean: {0}".format(feature))
    application_test = make_bool(application_test, feature)
    application_train = make_bool(application_train, feature)

Making Boolean: FLAG_OWN_CAR
Making Boolean: FLAG_OWN_REALTY
Making Boolean: FLAG_EMP_PHONE
Making Boolean: FLAG_WORK_PHONE
Making Boolean: FLAG_PHONE
Making Boolean: FLAG_EMAIL


### One-Hot Encode Non-Numeric Features

In [12]:
nnf = non_numeric_features['feature']
print(nnf)

# One-Hot Encode all of our non-numeric features
application_test = pd.get_dummies(application_test, columns=nnf)
application_train = pd.get_dummies(application_train, columns=nnf)

print("Training Set Columns: {0}".format(application_train.shape[1]))
print("Testing Set Columns: {0}".format(application_test.shape[1]))

# Ensure that train and test sets have the same number of columns
# from https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data/41339045
application_train,application_test = application_train.align(application_test, join='outer', axis=1, fill_value=0)

print("Aligned Training Set Columns: {0}".format(application_train.shape[1]))
print("Aligned Testing Set Columns: {0}".format(application_test.shape[1]))

0                    CODE_GENDER
1             NAME_CONTRACT_TYPE
2                NAME_TYPE_SUITE
3               NAME_INCOME_TYPE
4            NAME_EDUCATION_TYPE
5             NAME_FAMILY_STATUS
6              NAME_HOUSING_TYPE
7                OCCUPATION_TYPE
8     WEEKDAY_APPR_PROCESS_START
9        HOUR_APPR_PROCESS_START
10             ORGANIZATION_TYPE
11            FONDKAPREMONT_MODE
12           EMERGENCYSTATE_MODE
13                HOUSETYPE_MODE
14            WALLSMATERIAL_MODE
Name: feature, dtype: object
Training Set Columns: 269
Testing Set Columns: 265
Aligned Training Set Columns: 269
Aligned Testing Set Columns: 269


### Log Transform Skewed Numeric Features

In [13]:
# These features need log transformations but have negative values, and log() of negative values is undefined.
# We need to offset all the values in the feature such that they're positive. 
# We'll want to find the minimum value across the test and train dataset so that we offset consistently

offset = find_offset([application_train, application_test], 'DAYS_BIRTH')
application_train['DAYS_BIRTH'], offset = offset_negative_values(application_train, "DAYS_BIRTH", offset)
application_test['DAYS_BIRTH'], offset = offset_negative_values(application_train, "DAYS_BIRTH", offset)

offset = find_offset([application_train, application_test], 'DAYS_LAST_PHONE_CHANGE')
application_train['DAYS_LAST_PHONE_CHANGE'], offset = offset_negative_values(application_train, "DAYS_LAST_PHONE_CHANGE", offset)
application_test['DAYS_LAST_PHONE_CHANGE'], offset = offset_negative_values(application_train, "DAYS_LAST_PHONE_CHANGE", offset)

offset = find_offset([application_train, application_test], 'DAYS_EMPLOYED')
application_train['DAYS_EMPLOYED'], offset = offset_negative_values(application_train, "DAYS_EMPLOYED", offset)
application_test['DAYS_EMPLOYED'], offset = offset_negative_values(application_train, "DAYS_EMPLOYED", offset)

offset = find_offset([application_train, application_test], 'DAYS_ID_PUBLISH')
application_train['DAYS_ID_PUBLISH'], offset = offset_negative_values(application_train, "DAYS_ID_PUBLISH", offset)
application_test['DAYS_ID_PUBLISH'], offset = offset_negative_values(application_train, "DAYS_ID_PUBLISH", offset)

offset = find_offset([application_train, application_test], 'DAYS_REGISTRATION')
application_train['DAYS_REGISTRATION'], offset = offset_negative_values(application_train, "DAYS_REGISTRATION", offset)
application_test['DAYS_REGISTRATION'], offset = offset_negative_values(application_train, "DAYS_REGISTRATION", offset)

offset = find_offset([application_train, application_test], 'DAYS_EMPLOYED')
application_train['DAYS_EMPLOYED'], offset = offset_negative_values(application_train, "DAYS_EMPLOYED", offset)
application_test['DAYS_EMPLOYED'], offset = offset_negative_values(application_train, "DAYS_EMPLOYED", offset)


Offsetting Feature DAYS_BIRTH by 25230
Offsetting Feature DAYS_BIRTH by 25230
Offsetting Feature DAYS_LAST_PHONE_CHANGE by 4362.0
Offsetting Feature DAYS_LAST_PHONE_CHANGE by 4362.0
Offsetting Feature DAYS_EMPLOYED by 17913.0
Offsetting Feature DAYS_EMPLOYED by 17913.0
Offsetting Feature DAYS_ID_PUBLISH by 7198
Offsetting Feature DAYS_ID_PUBLISH by 7198
Offsetting Feature DAYS_REGISTRATION by 24673.0
Offsetting Feature DAYS_REGISTRATION by 24673.0
Offsetting Feature DAYS_EMPLOYED by 1
Offsetting Feature DAYS_EMPLOYED by 1


In [14]:
# Ensure all values are numeric, then log transform them
for feature in log_transform_features['feature']: 
        application_train[feature] = make_numeric(application_train, feature)
        application_test[feature] = make_numeric(application_test, feature)
    
        print("Performing Log Transformations on: {0}".format(feature))
        application_test = log_transform(application_test, feature)
        application_train = log_transform(application_train, feature)


Performing Log Transformations on: CNT_CHILDREN
Performing Log Transformations on: AMT_INCOME_TOTAL
Performing Log Transformations on: AMT_CREDIT
Performing Log Transformations on: AMT_ANNUITY
Performing Log Transformations on: AMT_GOODS_PRICE
Performing Log Transformations on: OWN_CAR_AGE
Performing Log Transformations on: OBS_30_CNT_SOCIAL_CIRCLE
Performing Log Transformations on: DEF_30_CNT_SOCIAL_CIRCLE
Performing Log Transformations on: OBS_60_CNT_SOCIAL_CIRCLE
Performing Log Transformations on: DEF_60_CNT_SOCIAL_CIRCLE
Performing Log Transformations on: DAYS_LAST_PHONE_CHANGE


### Remove Non-Numeric Values from Numeric Fields

In [15]:
for feature in numeric_features['feature']:
    print("Removing Non-Numeric Values from Feature {0}".format(feature))
    application_train[feature] = make_numeric(application_train, feature)
    application_test[feature] = make_numeric(application_test, feature)

Removing Non-Numeric Values from Feature REGION_POPULATION_RELATIVE
Removing Non-Numeric Values from Feature DAYS_BIRTH
Removing Non-Numeric Values from Feature DAYS_EMPLOYED
Removing Non-Numeric Values from Feature DAYS_REGISTRATION
Removing Non-Numeric Values from Feature DAYS_ID_PUBLISH
Removing Non-Numeric Values from Feature CNT_FAM_MEMBERS
Removing Non-Numeric Values from Feature AMT_REQ_CREDIT_BUREAU_HOUR
Removing Non-Numeric Values from Feature AMT_REQ_CREDIT_BUREAU_DAY
Removing Non-Numeric Values from Feature AMT_REQ_CREDIT_BUREAU_WEEK
Removing Non-Numeric Values from Feature AMT_REQ_CREDIT_BUREAU_MON
Removing Non-Numeric Values from Feature AMT_REQ_CREDIT_BUREAU_QRT
Removing Non-Numeric Values from Feature AMT_REQ_CREDIT_BUREAU_YEAR
Removing Non-Numeric Values from Feature APARTMENTS_AVG
Removing Non-Numeric Values from Feature APARTMENTS_MEDI
Removing Non-Numeric Values from Feature APARTMENTS_MODE
Removing Non-Numeric Values from Feature BASEMENTAREA_AVG
Removing Non-Numeri

### Scale Numeric Fields

In [16]:
'''
TODO: I think we'll have to: 
 - put the numeric features in their own DataFrame
 - scale them
 - drop the original features from the original data frames
 - add the dataframes with the scaled features back to the original dataframes
'''

 
numeric = [] # We need a list of all numeric features
ltf = log_transform_features['feature']
nf = numeric_features['feature']

# Construct a DataFrame with just the numeric features
#application_train_numeric = pd.DataFrame()
#application_test_numeric = pd.DataFrame()

# Initialize a scaler, then apply it to the features
# RobustScaler has superior support for data with outliers
scaler = StandardScaler()

for feature in ltf:
    #numeric.append("LOG_{0}".format(feature))
    numeric.append("{0}".format(feature))

for feature in nf: 
    numeric.append("{0}".format(feature))
    
for feature in numeric: 
       
    scaler = scaler.fit(application_train[numeric])
    application_train[numeric] = scaler.transform(application_train[numeric])
    application_test[numeric] = scaler.transform(application_test[numeric])

### Preview
Show our transformed dataset

In [17]:
# Show an example of a record with scaling applied
display(application_train.head(n = 5))

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_MOBIL,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUR_APPR_PROCESS_START_0,HOUR_APPR_PROCESS_START_1,HOUR_APPR_PROCESS_START_10,HOUR_APPR_PROCESS_START_11,HOUR_APPR_PROCESS_START_12,HOUR_APPR_PROCESS_START_13,HOUR_APPR_PROCESS_START_14,HOUR_APPR_PROCESS_START_15,HOUR_APPR_PROCESS_START_16,HOUR_APPR_PROCESS_START_17,HOUR_APPR_PROCESS_START_18,HOUR_APPR_PROCESS_START_19,HOUR_APPR_PROCESS_START_2,HOUR_APPR_PROCESS_START_20,HOUR_APPR_PROCESS_START_21,HOUR_APPR_PROCESS_START_22,HOUR_APPR_PROCESS_START_23,HOUR_APPR_PROCESS_START_3,HOUR_APPR_PROCESS_START_4,HOUR_APPR_PROCESS_START_5,HOUR_APPR_PROCESS_START_6,HOUR_APPR_PROCESS_START_7,HOUR_APPR_PROCESS_START_8,HOUR_APPR_PROCESS_START_9,HOUSETYPE_MODE_not specified,HOUSETYPE_MODE_org spec account,HOUSETYPE_MODE_reg oper account,HOUSETYPE_MODE_reg oper spec account,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUP

### Output
Save out our preprocessed data to temporary intermediate files

In [18]:
application_train.to_csv('data/tmp/application_train_standard_scaler_no_log_transformation_preprocessed.csv')
application_test.to_csv('data/tmp/application_test_standard_scaler_no_log_transformation_preprocessed.csv')